# Cargue de información

## Bibliotecas

In [10]:
# Descarga de las bibliotecas a utilizar

#%pip install nltk
#%pip install googletrans==4.0.0-rc1
#%pip install regex
#%pip install unidecode
#%pip install langdetect

In [2]:
# Importación de las bibliotecas a utilizar
import pandas as pd
from dateutil import parser
import gzip
import json
from pandas import json_normalize
import unicodedata

from langdetect import detect, LangDetectException

import regex
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [3]:
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Bases de datos - Rutas

In [4]:
# Se obtienen las rutas de los archivos
ruta_games = "../Data/originales/steam_games.json.gz"
ruta_reviews = "../Data/originales/user_reviews.json.gz"
ruta_items = "../Data/originales/users_items.json.gz"

## Funciones generales

In [65]:
# Se crea una función que abre los archivos json dañados y regresa una lista con los datos transformados.
def abrir_json_corrupto(ruta):
    '''A partir de la ruta de un archivo json corrupto comprimido en formato gzip, 
    abre el archivo y hace una limpieza de las líneas para acercarlas a un formato json'''
    # Abre el archivo JSON.GZIP para descomprimirlo y convertirlo en una lista.
    with gzip.open(ruta,"rt",encoding="utf-8", errors="replace") as archivo_original:
        lineas_items = archivo_original.readlines()

    # Se crea una lista donde se guardarán las líneas corregidas.
    data = []

    # Se revisa línea por línea y se reemplazan los datos que impiden que el JSON sea válido
    for linea in lineas_items:
        linea = linea.replace('"',"'")
        linea = linea.replace("True","true").replace("False","false")
        # linea = linea.replace("\': \'",'": "').replace("\', \'",'", "').replace("\': [{\'", '": [{"').replace("\': true, \'", '": true, "').replace("\': false, \'",'": false, "')
        linea = linea.replace("\':",'":').replace(", \'", ', "').replace("{\'", '{"').replace(": \'",': "').replace("\',",'",')
        #linea = linea.replace("\'}",'"}').replace("\\\'","'").replace("{\'", '{"').replace("': []",'": []')
        linea = linea.replace("\'}",'"}').replace("\\\'","'")
        linea = linea.strip('\n').strip("'")

        data.append(linea)

    # Se regresa la lista de las líneas después de la corrección realizada.
    return data

In [6]:
# Se crea una función con el propósito de verificar que los elementos sean json válidos
def es_json_valido(cadena):
    '''A partir de una cadena de texto, revisa si el elemento es json válido.'''
    try:
        json.loads(cadena)
        return True
    except ValueError:
        return False

# Se crea una función para validar la data de las listas de los elementos json transformados.
def validar_data(data):
    '''Valida la data de una lista, separando elementos json válidos de aquellos que no lo son.'''
    # Se crean las listas donde se guardaran los elementos separados.
    no_validos = []
    validos = []

    for elemento in data:
        if es_json_valido(elemento) == False:
            no_validos.append(elemento)
        else: validos.append(elemento)

    # Se imprime el número de elementos JSON no válidos
    print(f"EL total de elementos JSON no válidos de la data fue {len(no_validos)} de {len(data)} lo que equivale al {round((len(no_validos)/len(data)) * 100,2)} %")

    # Regresa la lista de datos válidos.
    return validos


In [7]:
# Se crea una función para cargar los elementos JSON corregidos a un dataframe de pandas.

def cargar_json_corregido(data):
    '''Toma una lista de elementos en formato JSON y los convierte a un dataframe de pandas'''
    #Convertir cada elemento de la lista a un diccionario
    diccionario = [json.loads(elemento) for elemento in data]

    # Combinar los diccionarios en uno solo
    data_json = json.dumps(diccionario, indent=2)
    
    # Se convierten a un dataframe de pandas.
    n_data = pd.read_json(data_json)

    return n_data


In [8]:
# Se define una función para revisar que los datos de las fechas sean válidos:
def es_fecha(fecha):
    try:
        n_fecha = parser.parse(fecha)
        return n_fecha
    except ValueError:
        return None

In [9]:
# Se crea una función para guardar los nuevos archivos

def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  # Toma el dataframe dado y lo convierte en un archivo json.
  archivo_json = DataFrame.to_json()
  # Comprime el archivo json en formato gzip.
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

# Apertura y revisión de las bases de datos

Información necesaria para las consultas: 
- Año y horas jugadas por juego y género del juego.
- Horas jugadas por usuario, género y años en los que jugó.
- Recomendaciones por juego (recommend y comentarios)
- Año y número de comentarios positivos, negativos o neutrales.

## Games

In [15]:
# Se abre el archivo de "Steam Games" y se convierte a un dataframe de pandas.
games = pd.read_json(ruta_games,compression="gzip",lines = True)
# Se obtiene la visualización de los primeros datos de la base.
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [16]:
# Se eliminan las filas cuyos datos sean todos nulos (ya que no nos proporcionan información sobre el juego).
games.dropna(how = "all", inplace = True)

In [17]:
# Se deciden eliminar las columnas "publisher", "tags","url","reviews_url","early_access", "title","specs","price","developer" ya que no proporcionan información relevante para las consultas a realizar o resulta redundante.
games.drop(columns=["publisher", "tags","url","reviews_url","early_access","title","specs","price","developer"],inplace=True)

In [19]:
# Se eliminan los valores vacíos de "release_date", ya que no nos proporcionan información relevante sobre el juego.
games = games.dropna(subset=["release_date"])

In [21]:
# Se revisa que los datos restantes para la columna "release_date" sean fechas válidas dejando la información en una nueva columna llamada "review_date"
games.insert(3,"review_date",games["release_date"].apply(es_fecha))

In [23]:
# Se elimina la columna de "release_date", se cambia el nombre de la nueva columna ("review_date") a "release_date" y se eliminan los datos nulos. 
games.drop(columns="release_date",inplace=True)
games.rename(columns={"review_date":"release_date"},inplace=True)
games.dropna(subset=["release_date"],inplace=True)

In [26]:
# Se eliminan los valores nulos de id.
games.dropna(subset=["id"],inplace=True)

In [28]:
# Se transforman los valores de id a entero
games["id"] = games["id"].astype(int)

In [30]:
# Se deciden eliminar todos los valores nulos de "genres"
games.dropna(subset=["genres"],inplace=True)

In [32]:
# Se aseguran que los datos estén en los formatos adecuados
games["release_date"] = pd.to_datetime(games["release_date"])
games["id"] = games["id"].astype(int)

In [33]:
# Se revisa la información obtenida.
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28660 entries, 88310 to 120443
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28660 non-null  object        
 1   app_name      28659 non-null  object        
 2   release_date  28660 non-null  datetime64[ns]
 3   id            28660 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 1007.6+ KB


In [31]:
# Se observa el dataframe obtenido.
games.head()

,genres,app_name,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540


In [35]:
# Se guarda la base de datos corregida
guardar_archivo(games,"../Data/corregida/r_games.json.gzip")

In [37]:
# Se observa que el dataset cargue correctamente.
games_n = pd.read_json("../Data/corregida/r_games.json.gzip",compression="gzip",convert_dates=['release_date'],date_unit="ms")
games_n.head()

,genres,app_name,release_date,id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540


## Reviews

### Limpieza bases de datos

In [66]:
# Se abre el archivo de reviews con la función para abrir json corruptos.
data_reviews = abrir_json_corrupto(ruta_reviews)

In [67]:
# Se imprimen las primeras líneas para revisar el resultado obtenido.
for linea in data_reviews[:5]:
  print(linea)

{"user_id": "76561197970982479", "user_url": "http://steamcommunity.com/profiles/76561197970982479", "reviews": [{"funny": "", "posted": "Posted November 5, 2011.", "last_edited": "", "item_id": "1250", "helpful": "No ratings yet", "recommend": true, "review": "Simple yet with great replayability. In my opinion does 'zombie' hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth 'zombie' splattering fun for the whole family. Amazed this sort of FPS is so rare."}, {"funny": "", "posted": "Posted July 15, 2011.", "last_edited": "", "item_id": "22200", "helpful": "No ratings yet", "recommend": true, "review": "It's unique and worth a playthrough."}, {"funny": "", "posted": "Posted April 21, 2011.", "last_edited": "", "item_id": "43110", "helpful": "No ratings yet", "recommend": true, "review": "Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so

In [68]:
# Se validan que los elementos sean json válidos.
json_reviews = validar_data(data_reviews)

# Debido a que los datos no corregidos adecuadamente se puede considerar como despreciables (pocos datos no válidos), se continúa con el proceso.

EL total de elementos JSON no válidos de la data fue 400 de 25799 lo que equivale al 1.55 %


In [70]:
# Se convierten los elementos a un dataframe de pandas.
reviews = cargar_json_corregido(json_reviews)
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [71]:
# Se elimina la columna de "user_url".
reviews.drop(columns=["user_url"],inplace=True)

In [72]:
# Se desagregan cada una de las reviews para cada usuario.
reviews = reviews.explode(column="reviews",ignore_index=True)

In [74]:
# Con el propósito de separar cada una de las reviews, se normaliza la columna "reviews" la cual se encuentra en formato JSON en un DataFrame nuevo.
x = json_normalize(reviews["reviews"])
# Se combina el DataFrame normalizado con el DataFrame original.
reviews = pd.concat([reviews, x], axis=1).drop('reviews', axis=1)

In [76]:
# Se eliminan las columnas que ya no son necesarias o útiles para las consultas que se realizarán.
reviews.drop(columns=["funny","helpful"],inplace=True)

In [78]:
# Se reemplazan los valores no deseados (espacios en blanco) en "posted"
reviews["posted"] = reviews["posted"].replace(r"^\s*$", pd.NA, regex=True)

# Se decide dejar la columna "posted", ya que se acerca más al año donde probablemente se hizo la compra del juego.
reviews.drop(columns=["last_edited"],inplace=True)

# Se separa la columna "posted" para poder obtener la fecha en una columna separada.
reviews[["estatus","date"]] = reviews["posted"].str.split(" ",n= 1,expand=True)

# Se elimina la columna estatus ya que no ofrece información relevante
reviews.drop(columns="estatus",inplace=True)

In [80]:
# Se transforma la columna date a formato string
reviews["date"] = reviews["date"].astype(str)
# Se revisan los datos con formato de fecha valida
reviews["date_valid"] = reviews["date"].apply(es_fecha)

In [82]:
# Se elimina la columna posted y date por no ofrecer información relevante
reviews.drop(columns=["posted","date"],inplace=True)

In [84]:
# Se cambia el nombre de "date_valid"
reviews.rename(columns={"date_valid":"posted"},inplace=True)

In [86]:
# Se eliminan las filas donde item_id, posted o recommend sea nulo
reviews = reviews.dropna(subset=["item_id","posted","recommend"])

In [88]:
# Se reemplazan las reviews vacías por un mensaje que diga "No review"
reviews["review"].replace([""," "],"No review",inplace = True)
# Se aseguran que todos los datos en "review" sean string.
reviews["review"] = reviews["review"].astype(str)

C:\Users\dmon2\AppData\Local\Temp\ipykernel_9824\410897906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["review"].replace([""," "],"No review",inplace = True)
C:\Users\dmon2\AppData\Local\Temp\ipykernel_9824\410897906.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["review"] = reviews["review"].astype(str)


In [87]:
# Se observa la base obtenida
reviews

,user_id,item_id,recommend,review,posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
57631,76561198312638244,70,True,a must have classic from steam definitely wort...,2024-07-10
57632,76561198312638244,362890,True,this game is a perfect remake of the original ...,2024-07-08
57633,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,2024-07-03
57634,LydiaMorley,730,True,:D,2024-07-20


In [89]:
# Se guarda la base de datos procesada hasta el momento
guardar_archivo(reviews,"../Data/procesamiento/reviews_pro.json.gzip")

In [90]:
reviews_p = pd.read_json("../Data/procesamiento/reviews_pro.json.gzip",compression="gzip",convert_dates=["posted"],date_unit="ms")
reviews_p.head()

,user_id,item_id,recommend,review,posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08


### Limpieza del texto de las reviews - análisis de emoción

In [92]:
#Se crea una función para limpiar la base de datos, buscando mantener la escritura
# de los idiomas no alfabéticos

def clean_text(text):
    # Reemplazo de caracteres comunes
    text = re.sub(r"10/10","amazing game", text)
    text = re.sub(r"<3","loved the game", text)
    text = re.sub(r":\)","good game", text)
    # Eliminar caracteres numéricos y de puntuación
    text = regex.sub(r"[^\p{L}p{Z}]", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    # Dejar el texto en minusculas
    text = text.lower()
    # Normalizar el texto a la forma NFKC (compatibilidad y composición)
    text = unicodedata.normalize('NFKC', text)

    return text

In [93]:
# Se hace limpieza de los datos en la columna review.
reviews_p["review_clean"] = reviews_p["review"].apply(clean_text)

In [95]:
# Se crea una función para detectar el lenguaje.
# En caso de que no sea capaz de detectarlo, no trae ningún valor.
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException as e:
        return None
    
# Se busca detectar los idiomas en las reviews. 
reviews_p["language"] = reviews_p["review_clean"].apply(detect_language)

In [97]:
# Se traen los datos de todas las reviews identificadas correctamente en inglés.
en_reviews = reviews_p[reviews_p["language"] == "en"]

In [98]:
# Se importa el modelo de análisis de sentimientos.
sia = SentimentIntensityAnalyzer()

In [100]:
# Se crea la definición del valor de respuesta que vamos a entender como un sentimiento:
  # Positivo
pos = 0.5
  # y negativo
neg = -0.5

# Definimos una función para analizar el sentimiento de la review.
def sentiment_analisys(texto):
  sentimiento = sia.polarity_scores(texto)["compound"]

  # Se define la variable de respuesta.
  r = int

  # Si el sentimiento se define como positivo, nos marca 2.
  if sentimiento >= pos:
    r = 2
  # Si el sentimiento se define como negativo, nos marca 0.
  elif sentimiento <= neg:
    r = 0
  # En caso contrario se define como neutral en 1.
  else:
    r = 1

  return r

In [101]:
# Se hace el análisis de sentimiento de cada una de las reviews
en_reviews["sentiment_analysis"] = en_reviews["review_clean"].apply(sentiment_analisys)

C:\Users\dmon2\AppData\Local\Temp\ipykernel_9824\2083790567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_reviews["sentiment_analysis"] = en_reviews["review_clean"].apply(sentiment_analisys)


In [102]:
# Se eliminan las columnas de "review_clean","review","language" ya que no representan infomración útil para las consultas.
en_reviews.drop(columns=["review_clean","review","language"],inplace=True)

C:\Users\dmon2\AppData\Local\Temp\ipykernel_9824\1537476710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_reviews.drop(columns=["review_clean","review","language"],inplace=True)


In [104]:
# Se obtiene información sobre la distribución del dataset en "recommend" y "sentiment_analisys".
print(en_reviews["recommend"].value_counts(normalize=True))
print(en_reviews["sentiment_analysis"].value_counts(normalize=True))

recommend
True     0.876766
False    0.123234
Name: proportion, dtype: float64
sentiment_analysis
2    0.585414
1    0.329513
0    0.085072
Name: proportion, dtype: float64


In [105]:
# Se asegura transformar el user_id a tipo str
en_reviews["user_id"] = en_reviews["user_id"].astype(str)

C:\Users\dmon2\AppData\Local\Temp\ipykernel_9824\1532270749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_reviews["user_id"] = en_reviews["user_id"].astype(str)


In [107]:
# Se revisa la información del dataset.
en_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43892 entries, 0 to 57633
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             43892 non-null  object        
 1   item_id             43892 non-null  int64         
 2   recommend           43892 non-null  bool          
 3   posted              43892 non-null  datetime64[ns]
 4   sentiment_analysis  43892 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int64(2), object(1)
memory usage: 1.7+ MB


In [106]:
# Se revisa la base de datos obtenida.
en_reviews

,user_id,item_id,recommend,posted,sentiment_analysis
0,76561197970982479,1250,True,2011-11-05,2
1,76561197970982479,22200,True,2011-07-15,1
2,76561197970982479,43110,True,2011-04-21,2
3,js41637,251610,True,2014-06-24,2
4,js41637,227300,True,2013-09-08,2
...,...,...,...,...,...
57629,76561198312638244,233270,True,2024-07-21,2
57630,76561198312638244,130,True,2024-07-10,2
57631,76561198312638244,70,True,2024-07-10,2
57632,76561198312638244,362890,True,2024-07-08,2


In [108]:
# Se guarda la base obtenida.
guardar_archivo(en_reviews,"../Data/corregida/r_reviews.json.gzip")

In [109]:
# Se revisa que la base cargue adecuadamente.
n_reviews = pd.read_json("../Data/corregida/r_reviews.json.gzip",compression="gzip",convert_dates=["posted"],date_unit="ms")
n_reviews.head()

,user_id,item_id,recommend,posted,sentiment_analysis
0,76561197970982479,1250,True,2011-11-05,2
1,76561197970982479,22200,True,2011-07-15,1
2,76561197970982479,43110,True,2011-04-21,2
3,js41637,251610,True,2014-06-24,2
4,js41637,227300,True,2013-09-08,2


## Items